In [ ]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Install required packages
    !pip install ase torch_geometric
    import torch
    from torch_geometric.data import Data, DataLoader

    dataset = "/content/drive/My Drive/Dataset"
else:
    dataset = "Dataset"

In [ ]:
import pandas as pd
path = f"{dataset}/absorption_mp_data.pkl"
the_data = pd.read_pickle(path)
the_data.head()


In [ ]:
a_structure = the_data["structure"][1]
for i in a_structure:
    #print(type(i))
    print([i.number, i.index, i.position, i.symbol])
    '''
    print(i.position)
    print(i.index)
    print(i.number)'''

In [ ]:
from ase import Atoms

atom_0 = a_structure[0].symbol
#, a_structure[0].position
print(atom_0)
#atom_1 = a_structure[1].symbol
#, a_structure[1].position
print(atom_1)
atom_2 = a_structure[2].symbol
print(atom_2)
atoms = Atoms([atom_0, atom_2])

distance = atoms.get_distance(0,1)
print(distance)

In [ ]:
from ase import Atoms
list_atoms = []
for i in a_structure:
    a_symbol = [i.symbol, i.position]
    list_atoms.append(a_symbol)
print(list_atoms)


bonds =list_atoms[0].get_bonds(list_atoms[1])
print(bonds)

In [ ]:
print(a_structure.symbols[0],(a_structure.symbols[1]))

In [ ]:
#the_data["energies"]
#energy_0 = np.size(the_data["energies"][0])
#abs_cof = np.size(the_data["absorption_coefficient"][0])
#ima_diel = np.size(the_data["imag_dielectric"][0])
rel_diel = np.size(the_data["real_dielectric"][0])

'''energy_0 = the_data["energies"][0]
i = 0
for x in energy_0:
    i = i +1

print(i)'''
rel_diel

In [ ]:
#from ase import Atoms
from ase.neighborlist import NeighborList, natural_cutoffs

# Example molecule: Water
#atoms = Atoms('H2O', positions=[[0, 0, 0], [0, 0.76, 0.58], [0, -0.76, 0.58]])
#atoms = Atoms(a_structure.symbols, a_structure.positions)
atoms = a_structure
# Step 1: Compute cutoffs for each atom (heuristic for bonding radius)
cutoffs = natural_cutoffs(atoms)

# Step 2: Build neighbor list
nl = NeighborList(cutoffs, self_interaction=False, bothways=True)
nl.update(atoms)

# Step 3: Loop over all atoms to get bonded neighbors (edges)
bonds = []
for i in range(len(atoms)):
    indices, offsets = nl.get_neighbors(i)
    for j, offset in zip(indices, offsets):
        if i < j:  # avoid double counting
            distance = atoms.get_distance(i, j, mic=True)
            bonds.append(((i, j), distance))

# Display edge (bond) info
for (i, j), dist in bonds:
    print(f"Bond between atom {i} ({atoms[i].symbol}) and {j} ({atoms[j].symbol}) - Distance: {dist:.2f} Å")